# Classification With Transformers

In [57]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ta
from tensorflow.keras import models
import plotly.graph_objects as go

In [72]:
data_train = pd.read_csv("https://raw.githubusercontent.com/Rerris/Technical_Analysis_Proyect/9a7d477bbed852e752101fe85ac928200ae7ebd5/data/aapl_5m_train.csv").dropna()
data_test = pd.read_csv("https://raw.githubusercontent.com/Rerris/Technical_Analysis_Proyect/9a7d477bbed852e752101fe85ac928200ae7ebd5/data/aapl_5m_test.csv").dropna()

In [73]:
data_train=data_train.drop(columns=["Timestamp","Datetime"])
data_test=data_test.drop(columns=["Timestamp","Datetime"])

In [74]:
data_train.head()

,Unnamed: 0,Gmtoffset,Open,High,Low,Close,Volume
0,0,0,133.570007,133.611602,132.389999,132.809997,6624663.0
1,1,0,132.750000,132.750000,131.809997,131.889999,2541553.0
2,2,0,131.500000,132.339996,131.500000,132.059997,2492415.0
3,3,0,132.000000,132.250000,131.899993,132.250000,1859131.0
4,4,0,132.000000,132.018096,131.520004,131.589996,1780105.0


In [75]:
rsi_results = pd.DataFrame(index=data_train.index)

for window in range(13, 49):
    rsi_indicator = ta.momentum.RSIIndicator(data_train['Close'], window)
    rsi_results['RSI_' + str(window)] = rsi_indicator.rsi()

ema_results = pd.DataFrame(index=data_train.index)
for span in range(50, 66):
    ema_indicator = ta.trend.EMAIndicator(data_train['Close'], span)
    ema_results['EMA_' + str(span)] = ema_indicator.ema_indicator()

# Unir los resultados al DataFrame original
data_train = pd.concat([data_train, rsi_results, ema_results], axis=1)

In [76]:
rsi_results_test = pd.DataFrame(index=data_test.index)
for window in range(13, 49):
    rsi_indicator_test = ta.momentum.RSIIndicator(data_test['Close'], window)
    rsi_results_test['RSI_' + str(window)] = rsi_indicator_test.rsi()

# Crear el objeto EMAIndicator para EMAs desde 50 hasta 65 para data_test
ema_results_test = pd.DataFrame(index=data_test.index)
for span in range(50, 66):
    ema_indicator_test = ta.trend.EMAIndicator(data_test['Close'], span)
    ema_results_test['EMA_' + str(span)] = ema_indicator_test.ema_indicator()

# Unir los resultados al DataFrame original data_test
data_test = pd.concat([data_test, rsi_results_test, ema_results_test], axis=1)

## Normalizing Data

In [77]:

train_mean = data_train.mean()
train_std = data_train.std()

# Normalizar data_train
norm_data_train = (data_train - train_mean) / train_std


test_mean = data_test.mean()
test_std = data_test.std()


norm_data_test = (data_test - test_mean) / test_std


## Generating our features

In [78]:
norm_data_test.columns

Index(['Gmtoffset', 'Open', 'High', 'Low', 'Close', 'Volume', 'RSI_13',
       'RSI_14', 'RSI_15', 'RSI_16', 'RSI_17', 'RSI_18', 'RSI_19', 'RSI_20',
       'RSI_21', 'RSI_22', 'RSI_23', 'RSI_24', 'RSI_25', 'RSI_26', 'RSI_27',
       'RSI_28', 'RSI_29', 'RSI_30', 'RSI_31', 'RSI_32', 'RSI_33', 'RSI_34',
       'RSI_35', 'RSI_36', 'RSI_37', 'RSI_38', 'RSI_39', 'RSI_40', 'RSI_41',
       'RSI_42', 'RSI_43', 'RSI_44', 'RSI_45', 'RSI_46', 'RSI_47', 'RSI_48',
       'EMA_50', 'EMA_51', 'EMA_52', 'EMA_53', 'EMA_54', 'EMA_55', 'EMA_56',
       'EMA_57', 'EMA_58', 'EMA_59', 'EMA_60', 'EMA_61', 'EMA_62', 'EMA_63',
       'EMA_64', 'EMA_65'],
      dtype='object')

In [101]:
# lags = 5

# X_train = pd.DataFrame()
# X_test = pd.DataFrame()

# for lag in range(lags):
#     X_train[f"Open_{lag}"] = norm_data_train.Open.shift(lag)
#     X_train[f"High_{lag}"] = norm_data_train.High.shift(lag)
#     X_train[f"Low_{lag}"] = norm_data_train.Low.shift(lag)
#     X_train[f"Close_{lag}"] = norm_data_train.Close.shift(lag)
    
#     X_test[f"Open_{lag}"] = norm_data_test.Open.shift(lag)
#     X_test[f"High_{lag}"] = norm_data_test.High.shift(lag)
#     X_test[f"Low_{lag}"] = norm_data_test.Low.shift(lag)
#     X_test[f"Close_{lag}"] = norm_data_test.Close.shift(lag)

X_train=norm_data_train.drop(columns=['Unnamed: 0', 'Gmtoffset', 'Open', 'High', 'Low', 'Close', 'Volume'])
X_test=norm_data_test.drop(columns=['Gmtoffset', 'Open', 'High', 'Low', 'Close', 'Volume'])

Y_train = (norm_data_train.Close * (1 + 0.01) < norm_data_train.Close.shift(-1)).astype(float)
Y_test = (norm_data_test.Close * (1 + 0.01) < norm_data_test.Close.shift(-1)).astype(float)

# Removing nans and last value
X_train = X_train.iloc[65:-1, :].values
X_test = X_test.iloc[65:-1, :].values

Y_train = Y_train.iloc[65:-1].values.reshape(-1, 1)
Y_test = Y_test.iloc[65:-1].values.reshape(-1, 1)

In [102]:
si=pd.DataFrame(X_train)
si

,0,1,2,3,4,5,6,7,8,9,...,42,43,44,45,46,47,48,49,50,51
0,0.744320,0.689228,0.633210,0.577078,0.521445,0.466753,0.413316,0.361320,0.310886,0.262076,...,-1.194984,-1.193845,-1.192706,-1.191569,-1.190435,-1.189303,-1.188174,-1.187050,-1.185930,-1.184815
1,0.218779,0.191652,0.161083,0.128003,0.093196,0.057299,0.020827,-0.015867,-0.052487,-0.088800,...,-1.195632,-1.194525,-1.193417,-1.192310,-1.191203,-1.190099,-1.188997,-1.187898,-1.186803,-1.185712
2,0.288206,0.258788,0.226159,0.191224,0.154744,0.117337,0.079494,0.041559,0.003813,-0.033524,...,-1.196143,-1.195068,-1.193992,-1.192915,-1.191839,-1.190763,-1.189689,-1.188617,-1.187547,-1.186481
3,0.260951,0.233047,0.201782,0.168076,0.132701,0.096287,0.059339,0.022210,-0.014809,-0.051490,...,-1.196671,-1.195628,-1.194582,-1.193535,-1.192487,-1.191439,-1.190392,-1.189345,-1.188300,-1.187258
4,-0.181146,-0.185848,-0.196014,-0.210590,-0.228650,-0.249402,-0.272176,-0.296489,-0.321909,-0.348070,...,-1.197795,-1.196772,-1.195746,-1.194718,-1.193688,-1.192658,-1.191627,-1.190598,-1.189569,-1.188542
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39089,1.234084,1.174303,1.116176,1.060157,1.006500,0.955337,0.906712,0.860636,0.817065,0.775922,...,-1.248308,-1.248221,-1.248134,-1.248048,-1.247963,-1.247879,-1.247796,-1.247715,-1.247635,-1.247557
39090,1.272509,1.211994,1.153105,1.096310,1.041875,0.989937,0.940548,0.893722,0.849422,0.807569,...,-1.247382,-1.247317,-1.247252,-1.247187,-1.247122,-1.247057,-1.246993,-1.246930,-1.246868,-1.246807
39091,1.302901,1.241697,1.182117,1.124635,1.069522,1.016921,0.966885,0.919432,0.874525,0.832087,...,-1.246453,-1.246410,-1.246366,-1.246322,-1.246276,-1.246231,-1.246186,-1.246140,-1.246095,-1.246051
39092,1.851819,1.786619,1.721551,1.657475,1.594968,1.534406,1.476023,1.420011,1.366443,1.315315,...,-1.244819,-1.244809,-1.244797,-1.244783,-1.244768,-1.244752,-1.244735,-1.244717,-1.244698,-1.244680


In [ ]:
39094

## Reshaping Tensors

In [104]:
features = X_train.shape[1]

X_train = X_train.reshape(-1, features, 1)
X_test = X_test.reshape(-1, features, 1)

## Transformer Model

In [105]:
def create_transformer(inputs, head_size, num_heads, dnn_dim):
    # Stacking layers
    l1 = tf.keras.layers.MultiHeadAttention(key_dim=head_size,
                                            num_heads=num_heads,
                                            dropout=0.2)(inputs, inputs)
    l2 = tf.keras.layers.Dropout(0.2)(l1)
    l3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)(l2)
    
    res = l3 + inputs
    
    # Traditional DNN
    l4 = tf.keras.layers.Conv1D(filters=4, kernel_size=1, activation="relu")(res)
    l5 = tf.keras.layers.Dropout(0.2)(l4)
    l6 = tf.keras.layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(l5)
    l7 = tf.keras.layers.LayerNormalization(epsilon=1e-6)(l6)
    return l7 + res

In [106]:
input_shape = X_train.shape[1:]

# Hyperparams
head_size = 256
num_heads = 4
num_transformer_blocks = 4
dnn_dim = 4
units = 128


# Defining input_shape as Input layer
input_layer = tf.keras.layers.Input(input_shape)

# Creating our transformers based on the input layer
transformer_layers = input_layer

for _ in range(num_transformer_blocks):
    # Stacking transformers
    transformer_layers = create_transformer(inputs=transformer_layers,
                                            head_size=head_size,
                                            num_heads=num_heads,
                                            dnn_dim=dnn_dim)

# Adding global pooling
pooling_layer = tf.keras.layers.GlobalAveragePooling1D(data_format="channels_last")\
                                                      (transformer_layers)

# Adding MLP layers
l1 = tf.keras.layers.Dense(units=128, activation="leaky_relu")(pooling_layer)
l2 = tf.keras.layers.Dropout(0.3)(l1)
l3 = tf.keras.layers.Dense(units=128, activation="leaky_relu")(l2)

# Last layer, units = 2 for True and False values
outputs = tf.keras.layers.Dense(units=2, activation="softmax")(l3)

# Model
model = tf.keras.Model(inputs=input_layer,
                       outputs=outputs,
                       name="transformers_classification")

metric = tf.keras.metrics.SparseCategoricalAccuracy()
adam_optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
#callbacks = [tf.keras.callbacks.EarlyStopping(monitor="loss",
#                                              patience=10,
#                                              restore_best_weights=True)]

In [107]:
model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=adam_optimizer,
    metrics=[metric],
)

In [108]:
model.fit(
    X_train,
    Y_train,
    epochs=15,
    batch_size=64,
    # callbacks=callbacks,
)

Epoch 1/40
611/611 ━━━━━━━━━━━━━━━━━━━━ 347s 539ms/step - loss: 0.6699 - sparse_categorical_accuracy: 0.5880
Epoch 2/40
611/611 ━━━━━━━━━━━━━━━━━━━━ 463s 758ms/step - loss: 0.6634 - sparse_categorical_accuracy: 0.5986
Epoch 3/40
104/611 ━━━━━━━━━━━━━━━━━━━━ 4:54 582ms/step - loss: 0.6662 - sparse_categorical_accuracy: 0.5951

KeyboardInterrupt: 

In [ ]:
# Save the Keras model
models.save_model(model, 'transformer_classifier.keras')

In [ ]:
train_predictions = model.predict(X_train)

1224/1224 [==============================] - 22s 18ms/step


In [ ]:
fig = go.Figure(data_train=[go.Candlestick(x=data_train.index,
                open=data_train['Open'],
                high=data_train['High'],
                low=data_train['Low'],
                close=data_train['Close'])])
predicted_prices =train_predictions

# Prediction
fig.add_trace(go.Scatter(x=data_train.index, y=predicted_prices.flatten(), mode='lines', name='Predicted Price',line=dict(color='blue')))

# Labels
fig.update_layout(title='Stock Price Prediction using Transformers',
                   xaxis_title='Time',
                   yaxis_title='Stock Price',
                   showlegend=True,
                   legend=dict(x=0, y=1))

fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()

## Test

In [ ]:
test_predictions = model.predict(X_test.values.reshape(-1, 31, 1))

In [ ]:
fig = go.Figure(data=[go.Candlestick(x=data_test.index,
                open=data_test['Open'],
                high=data_test['High'],
                low=data_test['Low'],
                close=data_test['Close'])])

# Prediction
fig.add_trace(go.Scatter(x=data_test.index, y=test_predictions.flatten(), mode='lines', name='Predicted Price',line=dict(color='blue')))

# Labels
fig.update_layout(title='Stock Price Prediction using Transformers',
                   xaxis_title='Time',
                   yaxis_title='Stock Price',
                   showlegend=True,
                   legend=dict(x=0, y=1))

fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()

## Backtesting

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model

# Load the saved model
model = load_model('transformer_classifier.keras')

# Initialize the portfolio
portfolio_value = 100000  # Initial portfolio value
cash = portfolio_value
stock = 0
history = []  # To keep track of buy/sell/hold actions

# Define other necessary variables for backtesting
time_steps = 1
take_profit = 1.01
stop_loss = 0.99
buying_price = 0  # Initialize the buying price
action = 'Hold'  # Initialize the action

# Perform predictions and execute backtesting
for i in range(len(X_test) - time_steps):
    # Prepare the test data and reshape it
    X_test_reshaped = X_test.iloc[i:i+time_steps].values.reshape(-1, 31, 1)
    
    # Predict the price of the next day
    predicted_price = model.predict(X_test_reshaped)[0][0]
    
    # Get the actual price of the next day
    actual_price = X_test.iloc[i+time_steps, 0]
    
    # Decide to buy, sell, or hold
    if predicted_price > actual_price:  # If the model predicts that the price will go up
        if cash > 0:
            # Buy only 5% of the portfolio value
            buy_value = min(cash, portfolio_value * 0.05)
            stock += buy_value / actual_price
            cash -= buy_value
            action = 'Buy'
            buying_price = actual_price
    elif predicted_price < actual_price or actual_price >= buying_price * take_profit or actual_price <= buying_price * stop_loss:  # If the model predicts that the price will go down or the take profit or stop loss is reached
        if stock > 0:
            # Sell only 5% of the portfolio value
            sell_value = min(stock * actual_price, portfolio_value * 0.05)
            stock -= sell_value / actual_price
            cash += sell_value
            action = 'Sell'
    else:
        action = 'Hold'
    
    # Calculate the portfolio value
    portfolio_value = cash + stock * actual_price
    
    # Save the action in the history
    history.append({
        'Day': i,
        'Action': action,
        'Cash': cash,
        'Stock': stock,
        'Portfolio Value': portfolio_value
    })

# Convert the history into a DataFrame
df_history = pd.DataFrame(history)


# %%
df_history

# %%
# Graficar el historial de la cartera
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_history['Day'], y=df_history['Portfolio Value'], mode='lines', name='Portfolio Value', line=dict(color='blue')))
fig.update_layout(title='Portfolio Value Over Time',
                   xaxis_title='Day',
                   yaxis_title='Portfolio Value',
                   showlegend=True,
                   legend=dict(x=0, y=1))
fig.show()

# %%
# Guardar la grafica como png
fig.write_image("portfolio_value_5D.png")